# Logins and users

Make sure you run the containers creation notebook first.

dbatools is **awesome** with Logins and Users

First we will set up the variables and connections to the containers

In [ ]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

## A story of a login

Run this cell below to set up the start

In [ ]:
New-DbaDatabase -SqlInstance $SQL1 -Name SockFactoryApp | Out-Null
$Password = ConvertTo-SecureString SockFactoryApp_User -AsPlainText
New-DbaLogin -SqlInstance $Sql1 -Login SockFactoryApp_User -SecurePassword $Password |Out-Null
New-DbaDbUser -SqlInstance $SQL1 -Database SockFactoryApp -Login SockFactoryApp_User -Username SockFactoryApp_User  | Out-Null
Remove-DbaLogin -SqlInstance $SQL1 -Login SockFactoryApp_User -Force   | Out-Null
$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('SockFactory_App', $sqladminpassword)
Invoke-DbaQuery -SqlInstance $SqlInstances[0] -SqlCredential $cred -Query "SELECT @@SERVER" -WarningAction SilentlyContinue
Invoke-DbaQuery -SqlInstance $SqlInstances[0] -SqlCredential $cred -Query "SELECT @@SERVER" -WarningAction SilentlyContinue
Invoke-DbaQuery -SqlInstance $SqlInstances[0] -SqlCredential $cred -Query "SELECT @@SERVER" -WarningAction SilentlyContinue
Invoke-DbaQuery -SqlInstance $SqlInstances[0] -SqlCredential $cred -Query "SELECT @@SERVER" -WarningAction SilentlyContinue


Write-Output "Setup finished"

# 3am Tuesday Morning

You receive a call out because the Sock Factory has shut down and 

> It's the database's fault  
The connection is failing

Amongst your troubleshooting steps (perhaps they could/should be in a notebook so the results get saved?) 
You look in the error log for failed logins

You can do this with dbatools 

In [ ]:
Get-DbaErrorLog -SqlInstance $sql1 -After (Get-Date).AddMinutes(-5) | Select LogDate,Source,Text  

No login?
Interesting.  

Then you remember a new replica was added to the Availability Group at the weekend. 

Maybe the DBA did not add the logins correctly

You need to check for the login

In [ ]:
Get-DbaLogin -SqlInstance $SQL1 -Login SockFactoryApp_User

No response means no login :-(

It's ok, just create a new login using the password from the secure password vault

In [ ]:
$Password = ConvertTo-SecureString SockFactoryApp_User -AsPlainText
New-DbaLogin -SqlInstance $Sql1 -Login SockFactoryApp_User -SecurePassword $Password

and check that it exists

In [ ]:
Get-DbaLogin -SqlInstance $SQL1 -Login SockFactoryApp_User

A quick email to the users to test and all will be well :-)

## Its not always that easy

The Users come back saying no socks are being made and it is still the database

You have a look in the error log (we will just simulate the error first)

In [ ]:
$sqladminPassword = ConvertTo-SecureString 'SockFactoryApp_User' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('SockFactoryApp_User', $sqladminpassword)
Invoke-DbaQuery -SqlInstance $SqlInstances[0] -SqlCredential $cred -Database SockFactoryApp -Query "PRINT 'All is Well'" -WarningAction SilentlyContinue

In [ ]:
Get-DbaErrorLog -SqlInstance $sql1 -After (Get-Date).AddMinutes(-5) | Select LogDate,Source,Text  

Hmmm 

> Failed to open the explicitly specified database 'SockFactoryApp'

Does the user exist?

In [ ]:
Get-DbaDbUser -SqlInstance $SQL1 -Database SockFactoryApp -ExcludeSystemUser

So the user exists but we can't login

Whats going on ?
.  
.  
.  
.  
.  
.  
.  
.  
.  
.  
.  
.  
  
  
    
     
       
         
           
             









If you guessed Orphaned user 
  
Let's check that

In [ ]:
Get-DbaDbOrphanUser -SqlInstance $sql1

We have an orphaned user :-(   
  
We can fix that  

In [ ]:
Repair-DbaDbOrphanUser -SqlInstance $sql1

Lets just a run a query as the user to make sure 

In [ ]:
$sqladminPassword = ConvertTo-SecureString 'SockFactoryApp_User' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('SockFactoryApp_User', $sqladminpassword)
Invoke-DbaQuery -SqlInstance $SqlInstances[0] -SqlCredential $cred -Database SockFactoryApp -Query "PRINT 'All is Well'" -MessagesToOutput

## Wash Up Meeting

The next day there was a wash-up meeting to find out why the outage happened and how to avoid it happening again.

The DBA mentions that dbatools has a command called `Copy-DbaLogins` which can be used to ensure that all logins are synced across instances.

Using a script like

````
try {
    $replicas = (Get-DbaAgReplica -SqlInstance $ENV:ComputerName).name -EnableException
}
catch {
    [System.Environment]::Exit(1)
}

foreach ($replica in $replicas) {
    Write-Output "For this replica $replica"
    $replicastocopy = $replicas | Where-Object { $_ -ne $replica }
    foreach ($Replicatocopy in $replicastocopy) {
        Write-Output "We will copy logins from $replica to $Replicatocopy"
        try {
          $output =  Copy-DbaLogin -Source $replica -Destination $replicatocopy -ExcludeSystemLogins -EnableException
        }
        catch {
            $error[0..5] | Format-List -Force | Out-String
            [System.Environment]::Exit(1)
        }
        if ($output.Status -contains 'Failed') {
            $error[0..5] | Format-List -Force | Out-String
            [System.Environment]::Exit(1)
            }
    }
}
````
In an agent job following the instructions at https://dbatools.io/agent will ensure that all logins are created on all replicas of an Availability Group

We don't have an Availability Group here but we can see what `Copy-DBaLogin` can do.

If we look at the logins on SQL2, we can see that SockFactoryApp_User does not exist

In [ ]:
Get-DbaLogin -SqlInstance $SQL2 | Select Name

We can run `Copy-DbaLogin` to copy the logins that do not exist from SQL1 to SQL2 (To drop and recreate logins you cna use the `-Force` parameter

In [ ]:
Copy-DbaLogin -Source $sql1 -Destination $SQL2 -ExcludeSystemLogins 

And when we check SQL2, we cna see the user

In [ ]:
Get-DbaLogin -SqlInstance $SQL2 | Select Name

# Exporting Users and Permissions to Excel

Making use of another PowerShell module `ImportExcel` we cna dynamically create Excel workbooks and format them.

To create an Excel workbook showing all of the users, logins and permissions on an instance - this code can be used. There is also an Azure Data Studio Notebook that will do this in my Notebooks Repo at [beard.media/Notebooks](beard.media/Notebooks)

Lets check for the required modules and install if needed

In [ ]:
#Check for modules and install
$Modules = 'dbatools', 'ImportExcel'

if ((Get-PsRepository -Name PSGallery).InstallationPolicy -ne 'Trusted') {
    Write-Output "The PowerShell Gallery is not trusted so I will trust it so that we can install the modules without interaction"
    try {
        Set-PsRepository -Name PSGallery -InstallationPolicy Trusted
    }
    catch {
        Write-Output " Failed to trust the gallery, trying to force it and also add package provider"
        Install-PackageProvider NuGet -Force
        Import-PackageProvider NuGet -Force
        Set-PSRepository -Name PSGallery -InstallationPolicy Trusted
    }
    
}
else {
    Write-Output "The PowerShell Gallery is trusted I will continue"
}
$Modules.ForEach{
    If (-not(Get-Module $psitem -ListAvailable)) {
        Write-Output "We don't have the $psitem module so we will install it"
        Install-Module $psitem -Scope CurrentUser -Force
    }
    else {
        Write-Output "We have the $psitem module already"
    }
}

In [ ]:
$ExcelDirectory = 'c:\temp\' # Alter this to the directory you want the file created
$SQlinstance = $SqlInstances[0]  # Alter this for the SQL Instance you want to get permissions for
$SqlCredential = $SqlCredential ## if you need a credential
Write-Output "Processing $sqlinstance"

$InstanceName = $SQlinstance.Split('\').Split('.').Split('\').Split(',') -join '_'
$ExcelFile = $ExcelDirectory + '\' + $InstanceName + '_Permssions_OneTab_' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'

Write-Output "    FileName is $ExcelFile"

$WorkSheetName = "Permissions"

    $excel = Get-DbaUserPermission -SqlInstance $sqlinstance -SqlCredential $SqlCredential | Export-Excel -Path $ExcelFile -WorksheetName $WorkSheetName -AutoSize -FreezeTopRow -AutoFilter -PassThru
  
    $rulesparam = @{
        Address   = $excel.Workbook.Worksheets[$WorkSheetName].Dimension.Address
        WorkSheet = $excel.Workbook.Worksheets[$WorkSheetName] 
        RuleType  = 'Expression'      
    }

    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("sysadmin",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("db_owner",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER LOGINS",$E1)))' -BackgroundColor PaleGreen 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER SECURABLES",$E1)))' -BackgroundColor PowderBlue 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB ROLE MEMBERS",$E1)))' -BackgroundColor GoldenRod 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB SECURABLES",$E1)))' -BackgroundColor BurlyWood 

    Close-ExcelPackage $excel


Write-Output ""
Write-Output "Finished Processing $sqlinstance"

$Excel = Open-ExcelPackage -Path $ExcelFile
Add-Worksheet -ExcelPackage $Excel -WorksheetName 'Title' -MoveToStart | Out-Null

$TitleSheet = $excel.Workbook.Worksheets['Title']
$Date = (Get-Date).ToLongDateString()
$TitleSheet.Cells["A1"].value = "This Worksheet shows the User Permissions for each database on $sqlinstance at $Date "
Set-ExcelRange -Worksheet $TitleSheet -Range "A:1" -Bold -FontSize 22 -Underline -UnderLineType Double

$TitleSheet.Cells["B3"].Value = "The Cells are colour coded as follows :-"
Set-ExcelRange -Worksheet $TitleSheet -Range "B3" -Bold -FontSize 18 
$TitleSheet.Cells["E5"].Value = "The Yellow Cells show members of the sysadmin role who have permission to do and access anything on the instance "
$TitleSheet.Cells["E6"].Value = "The Green Cells show the logins on the server"
$TitleSheet.Cells["E7"].Value = "The Blue Cells show the instance level permissions that have been granted to the logins"
$TitleSheet.Cells["E8"].Value = "The Orange Cells show the database role membership for the login"
$TitleSheet.Cells["E9"].Value = "The Brown Cells show specific database permissions that have been granted for the logins"

$TitleSheet.Cells["B11"].Value = "You can filter by Database on the Object column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C11" -FontSize 18

$TitleSheet.Cells["B12"].Value = "You can filter by User/Group/Login on the Member column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C12" -FontSize 18

Set-ExcelRange -Worksheet $TitleSheet -Range  "C5" -BackgroundColor Yellow
Set-ExcelRange -Worksheet $TitleSheet -Range  "C6" -BackgroundColor PaleGreen
Set-ExcelRange -Worksheet $TitleSheet -Range  "C7" -BackgroundColor PowderBlue 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C8" -BackgroundColor GoldenRod 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C9" -BackgroundColor BurlyWood 

Close-ExcelPackage $excel


Write-Output "                ###############                  "
Write-Output "        FileName is $ExcelFile            "
Write-Output "                ###############                  "
Write-Output ""

Invoke-Item $ExcelFile

# Imagine that you want to be able to recreate your logins

All you need to do is regularly run the below script to create a T-SQL file with all that is required for the login creation

In [ ]:
Export-dbalogin -SqlInstance $SQL1 -Path $FolderPath 

In [ ]:
notepad  C:\Users\mrrob\Documents\dbatoolsdemo\localhost,15592-20200301184218-login.sql